# Предсказание победителя в онлайн-игре

In [56]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn.cross_validation import KFold

import time
import datetime

## Подготовка обучающей выборки

In [27]:
#загрузим файл с признаками
features = pd.read_csv('./features.csv', index_col='match_id')
print features.shape
#загрузим файл с тестовой выборкой
X_test = pd.read_csv('./features_test.csv', index_col='match_id')

(97230, 108)


In [4]:
#посмотрим какие колонки стоит убрать. Очевидно, что они есть в тренировочной, но отсутствуют в тестовых выборках
columns_to_drop = list(set(features.columns) - set(X_test.columns))
print columns_to_drop

['tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant', 'tower_status_radiant', 'duration', 'radiant_win']


Анализ названий столбцов и их описание говорит нам, что целевая переменная находится в столбце 'radiant_win'.
т.к.: 1) принимает значения (0,1) 2) отсутствует в тестовой выборке 3) перевод соответсвует 4) в инструкции так написано

переменная - целевая, а <b>не признак</b>. Поэтому <b>переменная radiant_win НЕ используется как признак</b>, убрираем и запоминаем ее в результат<br>
(Уважаемый проверяющий, обратите, пожалуйста, внимание на формулировку вопроса. Правильный ответ - "нет". Спасибо)

In [197]:
#выбросим лишние признаки, которые заглядывали в будущее + выбросим целевую переменную radiant_win
data = features.drop(columns_to_drop,axis=1)
# Проверим, что данные для обучения и для тестирования - одинаковы
assert(all(data.columns == X_test.columns))
data.shape

(97230, 102)

Удалены из выборки поля tower_status_radiant, tower_status_dire, barracks_status_radiant, barracks_status_dir, теперь выпишем Y

In [5]:
#Выписываем итоговый столбец
#выделим Y в массив без названий строк и столбцов
Y=np.ravel(features[['radiant_win']]);

Посмотрим на то, много ли пропусков, и в каких столбцах.

In [42]:
total_rows = data.shape[0]
nan_dict = {'total': {'Всего не NaN': total_rows, 'Доля NaN': 0}}
for column_name in data.columns:
    not_nans = data[column_name].count()
    if not_nans < total_rows:
        nan_dict[column_name] = {'Всего не NaN': data[column_name].count(),
                                 'Доля NaN': round(1 - data[column_name].count()*1.0 / total_rows, 2)}
        
df = pd.DataFrame.from_dict(nan_dict, orient='index').sort_values(by='Доля NaN', ascending=False)
df

,Всего не NaN,Доля NaN
first_blood_player2,53243,0.45
radiant_flying_courier_time,69751,0.28
dire_flying_courier_time,71132,0.27
first_blood_player1,77677,0.20
first_blood_team,77677,0.20
first_blood_time,77677,0.20
dire_bottle_time,81087,0.17
radiant_bottle_time,81539,0.16
dire_first_ward_time,95404,0.02
radiant_first_ward_time,95394,0.02


In [41]:
#выводим список незаполненных столбцов для ответа на вопрос
list(df.axes[0])

['first_blood_player2',
 'radiant_flying_courier_time',
 'dire_flying_courier_time',
 'first_blood_player1',
 'first_blood_team',
 'first_blood_time',
 'dire_bottle_time',
 'radiant_bottle_time',
 'dire_first_ward_time',
 'radiant_first_ward_time',
 'dire_courier_time',
 'radiant_courier_time',
 'total']

Данные в столбцах first_blood_time, first_blood_team, first_blood_player1 соответствуют событию "first blood" соответственно их отсутствие говорит об отсутствии этого события за 5 минут.
first_blood_player2 отсутствует в примерно половине данных. возможно говорит о непричастности 2ого игрока в такой ситуации. Однако, возможно, это означает неполноту данных.
Данные в столбцах team_bottle_time, team_courier_time, team_flying_courier_time, team_first_ward_time(где team соответствует dire или radiant) говорят о времени использвания командой team определенных предметов. Соответсвенно отсутсвие говорит о не использовании командой этих предметов за первые 5 минут.


In [43]:
#В задании разрешено выбрать способ заполнить NAN ячейки ,поэтому:

#предполагаем, что если никто не сделал FB то его сделала некая 3я силаа...
data['first_blood_time'].fillna(0, inplace=True)
data['first_blood_team'].fillna(3, inplace=True)
data['first_blood_player1'].fillna(10, inplace=True)
data['first_blood_player2'].fillna(10, inplace=True)

#временные параметры пока просто занулим
data['radiant_bottle_time'].fillna(0, inplace=True)
data['radiant_courier_time'].fillna(0, inplace=True)
data['radiant_flying_courier_time'].fillna(0, inplace=True)
data['radiant_first_ward_time'].fillna(0, inplace=True)

data['dire_bottle_time'].fillna(0, inplace=True)
data['dire_courier_time'].fillna(0, inplace=True)
data['dire_flying_courier_time'].fillna(0, inplace=True)
data['dire_first_ward_time'].fillna(0, inplace=True)

## градиентный бустинг "в лоб"

попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки".<br>
обучение градиентного бустинга с проверкой качества на <b>кросс-валидаци по 5 блокам с shuffle=True</b>.
Используем в коде метрику качества (<b>AUC-ROC</b>)

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

In [57]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.grid_search import GridSearchCV

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from time import time

запустим градиентный бустинг "в лоб" с 30ю деревьями.
Будем оценивать качества алгоритма с помощью кросс-валидации по 5 блоками с shuffle=True

In [98]:
params={'n_estimators': 30}
grd = GradientBoostingClassifier(**params)

start_time = time()
kf = KFold(len(Y), n_folds=5,shuffle=True,random_state=42)
scores = cross_validation.cross_val_score(grd, data, y=Y, cv=kf,scoring ='roc_auc')

print 'ROC AUC score:', np.mean(scores)
print '\nTime elapsed:', round(time() - start_time, 2)

ROC AUC score: 0.689962217707

Time elapsed: 190.72


градиентный бустинг - работает. Попробуем определить отимальные параметры для 30 деревьев

In [59]:
X_train__, X_test__, y_train__, y_test__ = train_test_split(data, Y, test_size=0.8, random_state=57179)
n_estimators = 30
param_grid = {'learning_rate': [0.6],
              'max_depth': [3, 4],
              'min_samples_leaf': [1, 31, 37, 43],
              'max_features': [1.0, 0.3, 0.1]
              }
est = GradientBoostingClassifier(n_estimators=n_estimators, random_state=57179)
start_time = time()
gs_cv = GridSearchCV(est, param_grid, scoring='roc_auc', n_jobs=2).fit(X_train__, y_train__)
print(gs_cv.best_params_)
print('На обучение {} классификаторов ушло {:.2f}c'.format(np.prod([len(val) for key, val in param_grid.items()]), 
                                                           time() - start_time))

{'max_features': 0.1, 'learning_rate': 0.6, 'max_depth': 3, 'min_samples_leaf': 43}
На обучение 24 классификаторов ушло 146.75c


Попробуем соптимизировать вместе с деревьями

In [70]:
X_train__, X_test__, y_train__, y_test__ = train_test_split(data, Y, test_size=0.8, random_state=57179)
n_estimators = 30
param_grid = {'learning_rate': [0.6],
              'max_depth': [3, 4],
              'min_samples_leaf': [1, 31, 37, 43],
              'max_features': [1.0, 0.3, 0.1],
              'n_estimators': [10,20,30,40,50]
              }
est = GradientBoostingClassifier(random_state=57179)
start_time = time()
gs_cv = GridSearchCV(est, param_grid, scoring='roc_auc', n_jobs=2).fit(X_train__, y_train__)
print(gs_cv.best_params_)
print('На обучение {} классификаторов ушло {:.2f}c'.format(np.prod([len(val) for key, val in param_grid.items()]), 
                                                           time() - start_time))

{'max_features': 0.3, 'n_estimators': 50, 'learning_rate': 0.6, 'max_depth': 3, 'min_samples_leaf': 43}
На обучение 120 классификаторов ушло 827.19c


Обучим заново, классификатор с оптимальными параметрами

In [81]:
grd = GradientBoostingClassifier(**gs_cv.best_params_)

start_time = time()
kf = KFold(len(Y), n_folds=5,shuffle=True,random_state=42)
scores = cross_validation.cross_val_score(grd, data, y=Y, cv=kf,scoring ='roc_auc')

print 'ROC AUC score:', np.mean(scores)
print '\nTime elapsed:', round(time() - start_time, 2)

ROC AUC score: 0.706129464445

Time elapsed: 207.65


Попробуем посмотреть теперь эти параметры при разных деревьях

In [82]:
X_train__, X_test__, y_train__, y_test__ = train_test_split(data, Y, test_size=0.8, random_state=57179)
n_estimators = 30
param_grid = {'learning_rate': [0.6],
              'max_depth': [3],
              'min_samples_leaf': [43],
              'max_features': [0.3],
              'n_estimators': [10,20,30,40,50,60,70,100,200,500]
              }
est = GradientBoostingClassifier(random_state=57179)
start_time = time()
gs_cv = GridSearchCV(est, param_grid, scoring='roc_auc', n_jobs=2).fit(X_train__, y_train__)
print(gs_cv.best_params_)
print('На обучение {} классификаторов ушло {:.2f}c'.format(np.prod([len(val) for key, val in param_grid.items()]), 
                                                           time() - start_time))

{'max_features': 0.3, 'n_estimators': 50, 'learning_rate': 0.6, 'max_depth': 3, 'min_samples_leaf': 43}
На обучение 10 классификаторов ушло 217.18c


определено, что оптимальный параметр - 50 деревьев.
Рассмотрим подробнее

In [83]:
n_trees=[10,20,30,40,50,60,70,100,200,500]
kfold=5

for n_tr in n_trees :
    print ''
    print 'n_estimators',n_tr
    params = {'n_estimators': n_tr, 
              'learning_rate': 0.6,
              'max_depth': 3,
              'min_samples_leaf': 43,
              'max_features': 0.3, 'random_state': 241}
    grd = GradientBoostingClassifier(**params)

    start_time = datetime.datetime.now()
    kf = KFold(len(Y), n_folds=kfold,shuffle=True,random_state=42)
    scores = cross_validation.cross_val_score(grd, data, y=Y, cv=kf,scoring ='roc_auc')
    #print scores
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print scores.mean()


n_estimators 10
Time elapsed: 0:00:39.801000
0.682456285604

n_estimators 20
Time elapsed: 0:01:24.850000
0.695068808944

n_estimators 30
Time elapsed: 0:01:59.070000
0.700355516472

n_estimators 40
Time elapsed: 0:02:26.030000
0.703371631227

n_estimators 50
Time elapsed: 0:02:58.321000
0.705629521431

n_estimators 60
Time elapsed: 0:03:34.002000
0.706611177424

n_estimators 70
Time elapsed: 0:04:09.345000
0.707380084131

n_estimators 100
Time elapsed: 0:05:52.742000
0.708460205877

n_estimators 200
Time elapsed: 0:11:40.036000
0.708956694738

n_estimators 500
Time elapsed: 0:29:01.885000
0.702203552863


#### Выводы 
кросс-валидация для градиентного бустинга с 30 деревьями по подсчетам заняла 190.72 с. При этом удалось получилось качество 0.6899

В ходе эксперимента были определены оптимальные параметры. Так, эксперимент показал, качество при 50 деревьях - оптимальное.Т.е. при данных условиях удобно использовать 50 деревьев. Однако, при 30 деревьях и при 100 получилось примерно равным. А время не сильно больше. Дальнейшее увеличение количества деревьев приводит к резкому росту времени, но качество правктически не растет
Вообще ответ может зависеть от других параметров бустинка(например,от скорости обучения). При learning_rate=1 переобучение начинается как раз около 40-50 деревьев. Однако при learning_rate=0.2 обучение можно производить с пользой и до 300 деревьев.

Возможно стоит провести дополнительные исследования и манипуляции, чтобы ускорить обучение при увеличении количества деревьев.
Чтобы ускорить обучение, можно:
1) манипуляции с параметрами (например с max_features, max_depth и min_samples_leaf).
Так в ходе эксперимента показано, что используя параметры {'max_features': 0.1, 'learning_rate': 0.6, 'max_depth': 3, 'min_samples_leaf': 43} можно скорить обучение на четверть, причём повысить его качество до примерно 0.72 
2) не производилась никакая обработка признаков. Возможно, количество можно сократить без потери качества
3) Сокращение объектов. Подготовка к финальному этапу происходила на сокращенных данных с целью уменьшить время. При этом качество было таким же 


## Подход 2: логистическая регрессия

### попытка 2.1: обучение логистической регрессии на полном наборе признаков с проверкой качества на кросс-валидации и подбором оптимального коэффициента регуляризации

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

линейные алгоритмы чувствительны к масштабу признаков

In [87]:
#масштабируем признаки нашей подготовленной выборки
data_scale=StandardScaler().fit_transform(data)

Оценим качество логистической регрессии с помощью кросс-валидации. Подберем при этом лучший параметр регуляризации (C)

In [122]:
cross_val_folder = KFold(data_scale.shape[0], n_folds=5, shuffle=True, random_state=57179)
clf = LogisticRegression(penalty='l2', random_state=57179, n_jobs=1)
grid = {'C': np.power(10.0**0.333, np.arange(-7*3, 0*3))}
searcher = GridSearchCV(clf, grid, scoring='roc_auc', cv=cross_val_folder, n_jobs=1)
start_time = time()
searcher.fit(data_scale, Y)
print('На обучение {} классификаторов с кросс-валидацией по 5 блокам ушло {:.2f}c'.format(len(grid['C']),
                                                                                          time() - start_time))
scores = np.asarray(sorted([[score.parameters['C'], score.mean_validation_score] for score in searcher.grid_scores_]))
n_max = np.argmax(scores[:,1])

На обучение 21 классификаторов с кросс-валидацией по 5 блокам ушло 213.49c


In [123]:
print 'лучшее качество AUC_ROC={:.4} получилось при параметре регуляризации C={:.2}'.format(scores[n_max,1], scores[n_max,0])

лучшее качество AUC_ROC=0.7167 получилось при параметре регуляризации C=0.0047


Над всеми признаками в кросс-валидации по пяти блокам с параметром регуляризации C=0.004 получилось качество 0.716. Это немногим больше, чем базовый вариант градиентного бустинга с 30 деревьями
Логистическая регрессия работает неплохо потому, что зависимость силы команды в нашей задаче достаточно монотонно зависит от всех некатегориальных признаков и временных параметров, определяющих успешность команды
<b>Сравнение логистической регрессии и градиентного бустинга по качеству и времени работы</b>
Время выполнения одной кросс-валидации по 5 блокам составило около 14ти секунд. Это примерно в 14 раз быстрее, чем кросс-валидация по 30 деревьям.

In [126]:
# для проверки
clf = LogisticRegression(penalty='l2',random_state=57179, C=0.0047, n_jobs=1)
start_time = time()
kf = KFold(len(Y), n_folds=kfold,shuffle=True,random_state=42)
score = cross_validation.cross_val_score(clf, data_scale, Y, cv=kf, n_jobs=1,scoring='roc_auc').mean()
print 'Time elapsed:', time() - start_time
print score

Time elapsed: 14.5720000267
0.716608838087


### попытка 2.2: обучение логистической регрессии на числовых признаках с проверкой качества на кросс-валидации

Уберем категориальные столбцы

In [116]:
#Определим колонки, которые принадлежат героям
heros_columns = []
for r_or_d in ('r', 'd'):
    for i in range(1, 6):
        heros_columns.append('{}{}_hero'.format(r_or_d, i))

In [118]:
#выбросим их и еще одну
shortdata=data.drop(heros_columns + ['lobby_type'],axis=1)
# и сразу масштабируем
shortdata_scale=StandardScaler().fit_transform(shortdata)

In [127]:
clf = LogisticRegression(penalty='l2', C=0.0047)
start_time = time()
kf = KFold(len(Y), n_folds=kfold,shuffle=True,random_state=42)
score = cross_validation.cross_val_score(clf, shortdata_scale, Y, cv=kf, n_jobs=1,scoring='roc_auc').mean()
print 'Time elapsed:', time() - start_time
print score

Time elapsed: 14.4259998798
0.716619993175


Незначительно уменьшилось время и выросло качество. Т.е. отсутствие категориальных столбцов практически никак не влияет. Так как значения категориальных признаков бессмысленны в линейном смысле, то их веса были на несколько порядков меньше, поэтому они не оказывали особого влияния на прогноз.

### Попытка 2.3: обучение логистической регрессии на числовых признаках и признаках "мешка слов" с проверкой качества на кросс-валидации

Обработаем категориальные признаки: добавим данные о героях

In [134]:
herodata=data[heros_columns]
heroes = set(pd.unique(data[heros_columns].unstack()))
heroes |= set(pd.unique(X_test[heros_columns].unstack()))

print 'Встречающиеся герои:', sorted(heroes)
print 'Пропущенные герои из диапазона:', set(range(min(heroes), max(heroes)+1)) - heroes
print 'Длина диапазона:', len(list(range(min(heroes), max(heroes)+1)))
print 'Разных героев замечено:', len(heroes)

Встречающиеся герои: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 112]
Пропущенные герои из диапазона: set([24, 107, 108, 111])
Длина диапазона: 112
Разных героев замечено: 108


число уникальных героев - 108. Также можно убедиться, что всего их 112 (воспользоваться словарем героев).  Значит некоторые никогда не используются

In [136]:
# введем соответствие номеру героя номера столбца, чтобы правильно формировать мешок
d = dict(zip(heroes,range(len(heroes))))

In [138]:
#формируем мешок слов
X_pick = np.zeros((data.shape[0], len(heroes)))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        #print d[data.ix[match_id, 'r%d_hero' % (p+1)]-1
        X_pick[i, d[data.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        X_pick[i, d[data.ix[match_id, 'd%d_hero' % (p+1)]]] = -1

In [139]:
# клеим данные
finaldata=np.concatenate([X_pick, shortdata_scale], axis=1)

In [140]:
clf = LogisticRegression(penalty='l2', C=0.0047)
start_time = time()
kf = KFold(len(Y), n_folds=kfold,shuffle=True,random_state=42)
score = cross_validation.cross_val_score(clf, finaldata, Y, cv=kf, n_jobs=1,scoring='roc_auc').mean()
print 'Time elapsed:', time() - start_time
print score

Time elapsed: 19.8729999065
0.751176333232


Качество заметно возросло. Это объясняется адеккватным учетом категориальных признаков, т.к. конкретные герои действительно важны. 

In [141]:
# определим оптимальные праметры еще раз на новых данных
cross_val_folder = KFold(data_scale.shape[0], n_folds=5, shuffle=True, random_state=57179)
clf = LogisticRegression(penalty='l2', random_state=57179, n_jobs=1)
grid = {'C': np.power(10.0**0.333, np.arange(-7*3, 0*3))}
searcher = GridSearchCV(clf, grid, scoring='roc_auc', cv=cross_val_folder, n_jobs=1)
start_time = time()
searcher.fit(finaldata, Y)
print('На обучение {} классификаторов с кросс-валидацией по 5 блокам ушло {:.2f}c'.format(len(grid['C']),
                                                                                          time() - start_time))
scores = np.asarray(sorted([[score.parameters['C'], score.mean_validation_score] for score in searcher.grid_scores_]))
n_max = np.argmax(scores[:,1])

На обучение 21 классификаторов с кросс-валидацией по 5 блокам ушло 298.88c


In [142]:
print 'лучшее качество AUC_ROC={:.4} получилось при параметре регуляризации C={:.2}'.format(scores[n_max,1], scores[n_max,0])

лучшее качество AUC_ROC=0.752 получилось при параметре регуляризации C=0.047


## Построение предсказания

Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью ллучшей с точки зрения AUC-ROC на кросс-валидации модели.<br>
ROC AUC score лучшего Градиентного бустинга: 0.706<br>
ROC AUC score лучшей Логистической регресси: 0.752<br>
<br>
В нашем случае будем использовать логистическую регрессию при параметре регуляризации C=0.047

In [143]:
 C=0.047

### Предобработка тестовой выборки X_test

In [144]:
Xt=X_test.drop(heros_columns + ['lobby_type'],axis=1)

In [145]:
#предполагаем, что если никто не сделал FB то его сделала некая 3я силаа...
Xt['first_blood_time'].fillna(0, inplace=True)
Xt['first_blood_team'].fillna(3, inplace=True)
Xt['first_blood_player1'].fillna(10, inplace=True)
Xt['first_blood_player2'].fillna(10, inplace=True)

#временные параметры пока просто занулим
Xt['radiant_bottle_time'].fillna(0, inplace=True)
Xt['radiant_courier_time'].fillna(0, inplace=True)
Xt['radiant_flying_courier_time'].fillna(0, inplace=True)
Xt['radiant_first_ward_time'].fillna(0, inplace=True)

Xt['dire_bottle_time'].fillna(0, inplace=True)
Xt['dire_courier_time'].fillna(0, inplace=True)
Xt['dire_flying_courier_time'].fillna(0, inplace=True)
Xt['dire_first_ward_time'].fillna(0, inplace=True)

In [146]:
Xt_scale=StandardScaler().fit_transform(Xt)

In [147]:
Xt_pick = np.zeros((X_test.shape[0], len(heroes)))

for i, match_id in enumerate(X_test.index):
    for p in xrange(5):
        Xt_pick[i, d[X_test.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        Xt_pick[i, d[X_test.ix[match_id, 'd%d_hero' % (p+1)]]] = -1

In [148]:
TestData_fin=np.concatenate([Xt_pick, Xt_scale], axis=1)

## Предсказание победителя игры

In [149]:
#обучаем наш оптимальный классификатор
clf = LogisticRegression(C=C,penalty='l2', random_state=57179, n_jobs=1)
start_time = time()
clf.fit(finaldata, Y)
print('На обучение  ушло {:.2f}c'.format(time() - start_time))



На обучение  ушло 8.27c


Строим финальные предсказания с помощью функции <b>predict_proba</b>(то есть вычисляем оценки принадлежности классам)

In [153]:
y_pred = clf.predict_proba(TestData_fin)[:, 1]

In [196]:
print "Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?"
print np.min(y_pred),np.max(y_pred)

Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
0.00847951222194 0.996437272037


Сформируем массив данных для Kaggle

In [174]:
ans = pd.DataFrame(y_pred,  columns=['radiant_win'])
Mid= pd.DataFrame(X_test.axes[0],  columns=['match_id'])
fin=Mid.join(ans, how='left')

In [194]:
fin.head()

,match_id,radiant_win
0,6,0.825655
1,7,0.761304
2,10,0.193171
3,13,0.866309
4,16,0.246961


Сравнение минимального и максимального значений вероятности и вид списка ответов показывают, что
1) вероятности разные
2) на интервале [0,1]

In [195]:
fin.to_csv('kaglle.csv', sep=',', header=True, index=False)